In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("dirty_cafe_sales.csv")

In [4]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
df.drop_duplicates(inplace = True)

In [6]:
df.shape

(10000, 8)

In [7]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [9]:
sentinels = ['ERROR','UNKNOWN','N/A','NA','NULL','']
df.replace(sentinels, np.nan, inplace=True)

In [10]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [11]:
for col in ['Total Spent', 'Price Per Unit', 'Quantity']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)
    df[col] = df[col].astype(float)

In [12]:
for col in ['Payment Method', 'Location']:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)

In [13]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,8.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Digital Wallet,Takeaway,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [14]:
df.isnull().sum()

Transaction ID        0
Item                969
Quantity              0
Price Per Unit        0
Total Spent           0
Payment Method        0
Location              0
Transaction Date    460
dtype: int64

In [15]:
mode_item = df['Item'].mode()[0]
df['Item'].fillna(mode_item, inplace=True)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
mode_date = df['Transaction Date'].mode()[0]
df['Transaction Date'].fillna(mode_date, inplace=True)
for col in ['Payment Method', 'Location']:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)

In [16]:
df['computed_total'] = df['Quantity'] * df['Price Per Unit']
df['total_diff'] = (df['Total Spent'] - df['computed_total']).abs()
tol = 1e-6
df['Total Spent'] = np.where(df['total_diff'] <= tol, df['Total Spent'], df['computed_total'])
df.drop(columns=['computed_total','total_diff'], inplace=True)

In [17]:
cols = ['Transaction ID','Item','Quantity','Price Per Unit','Total Spent','Payment Method','Location','Transaction Date']

print(df.isnull().sum())

unk_count = (df[cols] == 'UNKNOWN').sum().sum()
err_count = (df[cols] == 'ERROR').sum().sum()

print("Count of 'UNKNOWN' across selected columns:", unk_count)
print("Count of 'ERROR' across selected columns:", err_count)

display(df.head())

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64
Count of 'UNKNOWN' across selected columns: 0
Count of 'ERROR' across selected columns: 0


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Digital Wallet,Takeaway,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
